In [ ]:
import os
base_dir = 'data/trainset_11classes_0_00000'
train_folder = os.path.join(base_dir, 'train')
val_folder = os.path.join(base_dir, 'val')

print("Directory, Number of files")
for root, subdirs, files in os.walk(base_dir):
    print(root, len(files))

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 4
ncols = 4

pic_index = 0
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import re

bs = 32

image_size = (299, 299)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

print("Preparing generator for train dataset")
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

print("Preparing generator for validation dataset")
val_generator = val_datagen.flow_from_directory(
    directory= val_folder, 
    target_size=image_size,
    batch_size=bs,
    class_mode='categorical')


from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D

model = Sequential()
model.add(ZeroPadding2D((1,1), input_shape=(299, 299, 3)))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(11, activation='relu'))

model.summary()

from keras import optimizers

model.compile(loss='mean_squared_error',
              optimizer=optimizers.SGD(lr=0.5),
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint

bestValidationCheckpointer = ModelCheckpoint('train_model.hdf5', monitor='val_acc', save_best_only=True, verbose=1)

history = model.fit_generator(
        train_generator, # train generator has 973 train images
        steps_per_epoch=train_generator.samples // bs + 1,
        epochs=20,
        validation_data=val_generator, # validation generator has 253 validation images
        validation_steps=val_generator.samples // bs + 1,
        callbacks=[bestValidationCheckpointer],
        verbose=1
)

from keras.models import load_model

model_path = 'saved_model.hdf5'
model = load_model( model_path )

val_generator.reset()

scores = model.evaluate_generator(val_generator, steps=val_generator.samples // val_generator.batch_size + 1, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])



